In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import torch

In [2]:
# load data
df = pd.read_csv(" ") # path to the data

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")

In [5]:
label_list = {0: 'non-hate', 1: 'hate'}

In [6]:
def prediction(text):   
    inputs = tokenizer(text, return_tensors="pt")
    
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    predicted_class_label = label_list[predicted_class_id]
    confidence = torch.softmax(logits, dim=1)[0].tolist()
    max_confidence = max(confidence)

    return predicted_class_label, max_confidence

In [8]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    sentence = row['text']
    # print(sentence)
    pred_label, confidence = prediction(sentence)
    df.loc[index, 'prediction'] = pred_label
    df.loc[index, 'confidence'] = confidence   

  0%|          | 0/4000 [00:00<?, ?it/s]

In [12]:
df.groupby('prediction').count()

,index,text,original_text,category,quarter,confidence
prediction,,,,,,
hate,1258,1258,1258,1258,1258,1258
non-hate,2742,2742,2742,2742,2742,2742


In [14]:
df.to_csv('../data/roberta_output.csv', index=False)